## k_fold.ipynb

### Valutazione di modelli con grid search e random search tramite K-fold cross validation
> Questo notebook fornisce un ambiente interattivo per l'esplorazione e l'addestramento di reti neurali per il riconoscimento delle cifre del dataset MNIST, utilizzando la libreria custom messa a disposizione dagli autori. Inoltre, fornisce un framework completo per eseguire la k-fold cross validation e la selezione degli iperparametri,

Autori:
- Alessandro Trincone
- Mario Gabriele Carofano

### Importazione di moduli e librerie
> Per il corretto funzionamento del notebook, è necessario importare alcuni moduli e librerie, tra cui:
> - la libreria `numpy` per l'utilizzo di strutture dati e funzioni matematiche efficienti;
> - il modulo `NeuralNetwork` per costruire reti neurali tramite la libreria custom messa a disposizione dagli autori.

In [ ]:
import auxfunc
import constants
from artificial_neural_network import NeuralNetwork
from training_report import TrainingReport
from training_params import TrainingParams
import dataset_functions as df
import plot_functions as pf

In [ ]:
import numpy as np
import pprint
import time
from datetime import datetime
import copy
import gc

In [ ]:
import pandas as pd
import random
import os

Con questo quaderno, gli utenti possono:

### Eseguire la K-fold cross validation
> Questa tecnica di validazione può essere utilizzata per la selezione degli iper-parametri di addestramento come:
> - `eta_minus` e `eta_plus`, specifici dell'algoritmo di aggiornamento dei pesi RPROP;
> - `l_sizes`, cioè la lista delle dimensioni dei layer interni della rete neurale.

In [ ]:
def k_fold_cross_validation(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
        l_sizes : list[int] = constants.DEFAULT_HIDDEN_LAYER_NEURONS,
        params : TrainingParams = None
) -> list[dict]:
    
    """
        E' una tecnica di validazione che utilizza una parte indipendente del training set per la fase di validazione. Si utilizza per la selezione degli iper-parametri che restituiscono il minor errore di validazione sull'addestramento del modello.

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.
        -   l_sizes : e' una lista contenente le dimensioni di uno o piu' hidden layer e dell'output layer della rete neurale.
        -   params : e' un oggetto della classe TrainingParams che contiene alcuni iperparametri per la fase di addestramento della rete neurale.

        Returns:
        -   k_fold_report : una lista di metriche di valutazione relative ai valori di errore e di accuracy di validazione ottenuti durante le fasi di addestramento dei modelli sulle diverse fold.
    """

    k_fold_report = []

    Xfolds, Yfolds = df.split_dataset(Xtrain, Ytrain, k)

    for i in range(k):

        print(f"\nFold {i+1} di {k}")

        # Tutti gli altri parametri di NeuralNetwork sono inizializzati con i valori default
        net = NeuralNetwork(l_sizes=l_sizes)

        training_fold = np.concatenate([fold for j, fold in enumerate(Xfolds) if j != i])
        training_labels = np.concatenate([fold for j, fold in enumerate(Yfolds) if j != i])
        validation_fold = Xfolds[i]
        validation_labels = Yfolds[i]

        history_report = net.train(
            training_fold, training_labels,
            validation_fold, validation_labels,
            params
        )

        train_errs = [r.validation_error for r in history_report]
        train_accs = [r.validation_accuracy for r in history_report]

        k_fold_report.append({
            "Fold"      : i+1,
            "E_value"   : net.training_report.validation_error,
            "E_min"     : np.min(train_errs),
            "E_max"     : np.max(train_errs),
            "A_value"   : net.training_report.validation_accuracy,
            "A_min"     : np.min(train_accs),
            "A_max"     : np.max(train_accs)
        })

        del net
        del training_fold, training_labels
        del validation_fold, validation_labels
        gc.collect()

        if constants.DEBUG_MODE:
            break

    # end for i
    
    return k_fold_report

# end

### Eseguire la grid search per il tuning degli iper-parametri

In [ ]:
def grid_search_cv(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
) -> pd.DataFrame:
    
    """
        Ricerca la miglior combinazione di iper-parametri in uno spazio dei parametri ben delimitato da una griglia di valori (da qui, il nome Grid Search) addestrando una nuova rete neurale per ogni possibile combinazione di tale griglia.

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.

        Returns:
        -   Restituisce un pandas.Dataframe contenente tutte le combinazioni di iper-parametri che sono stati verificati da quest'operazione di ricerca e le rispettive metriche di valutazione. 
    """

    # Il valore tipico per "eta minus" e' compreso tra 0.5 e 0.9.
    # Valori più vicini a 0.5 riducono il passo di aggiornamento in modo più aggressivo, migliorando la stabilita', ma potenzialmente rallentando la convergenza.
    # Valori più vicini a 0.9 sono meno aggressivi, permettendo una convergenza più rapida ma con il rischio di instabilita'.
    eta_minus_values = [0.5, 0.7, 0.9]

    # Il valore tipico per "eta plus" e' compreso tra 1.2 e 1.5.
    # Valori più vicini a 1.2 incrementano il passo di aggiornamento in modo più moderato, migliorando la stabilita' e riducendo il rischio di oscillazioni.
    # Valori più vicini a 1.5 incrementano il passo di aggiornamento in modo più aggressivo, accelerando la convergenza ma aumentando il rischio di instabilita'.
    eta_plus_values = [1.2, 1.3, 1.5]

    # Per problemi di classificazione (es. riconoscimento di cifre scritte a mano), un intervallo comune per il numero di neuroni è da 10 a 100 per uno o due hidden layer.
    # Tuttavia, siccome l'input ha alta dimensionalita', potrebbe essere necessario aumentare il numero di neuroni per catturare le caratteristiche importanti. Ad esempio, per immagini 28x28, l'hidden layer puo' avere un numero di neuroni compreso tra 128 e 512.
    hidden_layer_values = [128, 256, 512]

    # Si inizializza un "DataFrame" per la raccolta delle metriche di valutazione.
    stats = pd.DataFrame({
        "Eta minus"     : [],
        "Eta plus"      : [],
        "Hidden layer"  : [],
        "Mean error"    : [],
        "Std error"     : [],
        "Mean accuracy" : [],
        "Std accuracy"  : []
    })

    # Si calcolano tutte le possibili combinazioni di iper-parametri.
    combs = [
        (x, y, z)
        for x in eta_minus_values
        for y in eta_plus_values
        for z in hidden_layer_values
    ]

    # Questo ciclo "for" esamina tutte le possibili combinazioni di iper-parametri.
    for i, (em, ep, hl) in enumerate(combs):

        print(f"\nCombinazione {i+1}:\n\tEta minus = {em},\n\tEta plus = {ep},\n\tHidden layer = {hl}\n")

        # Si applica la tecnica di validazione "K-fold cross validation" per valutare la combinazione di iper-parametri corrente.
        k_fold_report = k_fold_cross_validation(
            out_directory,
            Xtrain, Ytrain, k,
            [hl, 10],
            params=TrainingParams(
                eta_minus=em,
                eta_plus=ep
            )
        )

        fold_errs = [r['E_value'] for r in k_fold_report]
        fold_accs = [r['A_value'] for r in k_fold_report]

        # Si crea un dizionario per raccogliere tutte le metriche di valutazione riguardo la rete appena addestrata sulla combinazione di iperparametri corrente.
        new_row = {
            "Eta minus"     : em,
            "Eta plus"      : ep,
            "Hidden layer"  : hl,
            "Mean error"    : np.mean(fold_errs),
            "Std error"     : np.std(fold_errs),
            "Mean accuracy" : np.mean(fold_accs),
            "Std accuracy"  : np.std(fold_accs)
        }

        # Si aggiunge il dizionario come nuova riga alla tabella generale.
        stats = pd.concat([stats, pd.DataFrame([new_row])], ignore_index=True)

        # Si ripristina l'indice per mantenere l'ordinamento cronologico.
        stats = stats.reset_index(drop=True)

        pf.plot_search_report(out_directory, "Grid Search", k_fold_report, stats.iloc[-1])
    
    # end for i, (em, ep, hl)

    # Si ordina il DataFrame in modo non decrescente per i valori della colonna 'Mean Error'.
    stats = stats.sort_values(by=['Mean error'])

    formatted_stats = stats.style.format({
        "Eta minus"     : '{:.5f}',
        "Eta plus"      : '{:.5f}',
        "Hidden layer"  : '{:.0f}',
        "Mean error"    : '{:.5f}',
        "Std error"     : '{:.5f}',
        "Mean accuracy" : '{:.2%}',
        "Std accuracy"  : '{:.2%}'
    })
    
    # Si visualizzano le statistiche raccolte in una tabella.
    display(formatted_stats)

    # Si salva il contenuto della tabella in un file .csv.
    os.makedirs(out_directory, exist_ok=True)
    formatted_stats.to_string(out_directory + 'grid_search_stats.csv', delimiter=',')

    return stats

# end

# RIFERIMENTI
# Grid Search vs. Random Search : https://www.youtube.com/watch?v=G-fXV-o9QV8
# https://saturncloud.io/blog/how-to-insert-a-row-to-pandas-dataframe/
# https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-table-style/
# https://stackoverflow.com/questions/75956209/error-dataframe-object-has-no-attribute-append
# https://stackoverflow.com/questions/17006641/single-line-nested-for-loops
# https://stackoverflow.com/questions/20937538/how-to-display-pandas-dataframe-of-floats-using-a-format-string-for-columns

### Eseguire la random search per il tuning degli iper-parametri

In [ ]:
def random_search_cv(
        out_directory : str,
        Xtrain : list[np.ndarray],
        Ytrain : list[np.ndarray],
        k : int = constants.DEFAULT_K_FOLD_VALUE,
        r : int = constants.DEFAULT_RANDOM_COMBINATIONS
) -> pd.DataFrame:
    
    """
        Ricerca la miglior combinazione di iper-parametri campionando in modo casuale (da qui, il nome Random Search) all'interno dell'intero spazio dei parametri un numero limitato di combinazioni. Quindi, per ognuna di queste combinazioni addestra una nuova rete neurale.

        Parameters:
        -   out_directory : la directory di output dove salvare i grafici della k-fold cross validation.
        -   Xtrain : la matrice di esempi da classificare.
        -   Ytrain : la matrice di etichette corrispondenti agli esempi (ground truth).
        -   k : e' un numero intero che indica in quante fold dividere il training set.
        -   r : e' un numero intero che indica il numero di combinazioni di iperparametri da testare.

        Returns:
        -   Restituisce un pandas.Dataframe contenente tutte le combinazioni di iper-parametri che sono stati verificati da quest'operazione di ricerca e le rispettive metriche di valutazione.
    """

    stats = pd.DataFrame({
        "Eta minus"     : [],
        "Eta plus"      : [],
        "Hidden layer"  : [],
        "Mean error"    : [],
        "Std error"     : [],
        "Mean accuracy" : [],
        "Std accuracy"  : []
    })

    # Questo ciclo "for" verifica solo "r" combinazioni di iper-parametri.
    for i in range(r):

        em = np.random.uniform(low=0.5, high=0.9)
        ep = np.random.uniform(low=1.2, high=1.5)
        hl = np.random.randint(low=128, high=512)
        
        print(f"\nCombinazione {i+1}:\n\tEta minus = {em},\n\tEta plus = {ep},\n\tHidden layer = {int(hl)}\n")

        k_fold_report = k_fold_cross_validation(
            out_directory,
            Xtrain, Ytrain, k,
            [hl, 10],
            params=TrainingParams(
                eta_minus=em,
                eta_plus=ep
            )
        )

        fold_errs = [r['E_value'] for r in k_fold_report]
        fold_accs = [r['A_value'] for r in k_fold_report]

        new_row = {
            "Eta minus"     : em,
            "Eta plus"      : ep,
            "Hidden layer"  : hl,
            "Mean error"    : np.mean(fold_errs),
            "Std error"     : np.std(fold_errs),
            "Mean accuracy" : np.mean(fold_accs),
            "Std accuracy"  : np.std(fold_accs)
        }

        stats = pd.concat([stats, pd.DataFrame([new_row])], ignore_index=True)
        stats = stats.reset_index(drop=True)

        pf.plot_search_report(out_directory, "Random Search", k_fold_report, stats.iloc[-1])

    # end for i

    stats = stats.sort_values(by=['Mean error'])
    formatted_stats = stats.style.format({
        "Eta minus"     : '{:.5f}',
        "Eta plus"      : '{:.5f}',
        "Hidden layer"  : '{:.0f}',
        "Mean error"    : '{:.5f}',
        "Std error"     : '{:.5f}',
        "Mean accuracy" : '{:.2%}',
        "Std accuracy"  : '{:.2%}'
    })
    
    display(formatted_stats)

    os.makedirs(out_directory, exist_ok=True)
    formatted_stats.to_string(out_directory + 'random_search_stats.csv', delimiter=',')

    return stats

# end

### Valutare le prestazioni e le predizioni della rete neurale
> Grazie alla libreria messa a disposizione dagli autori del notebook, è possibile addestrare e valutare le prestazioni (nonché le predizioni) della rete neurale addestrata tramite specifiche funzionalità per la visualizzazione ed il salvataggio di report nella forma di tabelle, grafici di diverso tipo e anche immagini.

> In questo esempio di utilizzo:
> - Si carica il MNIST training set e il MNIST test set in due strutture dati separate.
> - Si memorizza in 'out_directory' la directory di output dove salvare i grafici richiesti.
> - Si applica la Grid Search K-Fold Cross Validation separando il MNIST training set in due sottoinsiemi distinti e separati (training e validation)
> - Si applica la Random Search K-Fold Cross Validation (applicando la suddivisione come sopra).
> - Si addestra una nuova rete neurale sulla miglior combinazione di iperparametri ottenuta sull'intero MNIST training set.
> - Si verificano le predizioni della rete neurale sul MNIST test set.

In [ ]:
idTrain, Xtrain, Ytrain, idTest, Xtest, Ytest = df.loadDataset(constants.COPPIE_TRAINING, constants.COPPIE_TEST)

In [ ]:
out_directory = constants.OUTPUT_DIRECTORY + datetime.now().strftime(constants.OUTPUT_DATE_TIME_FORMAT) + "/"

In [ ]:
print(f"\nValutazione con grid search iniziata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
start_time = time.time()

# Esecuzione della grid search.
grid_stats = grid_search_cv(out_directory, Xtrain, Ytrain)
grid_best_params_idx = grid_stats['Mean error'].idxmin()

# Visualizzazione della miglior combinazione di iper-parametri.
print(f"\nMiglior combinazione:\n\tIndex = {grid_best_params_idx},\n\tEta minus = {grid_stats['Eta minus'][grid_best_params_idx]:.5f},\n\tEta plus = {grid_stats['Eta plus'][grid_best_params_idx]:.5f},\n\tHidden layer = {grid_stats['Hidden layer'][grid_best_params_idx]:.0f}\n")

# Visualizzazione del tempo impiegato.
end_time = time.time()
tot_time = end_time - start_time

print(f"\nValutazione con grid search completata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
print(f"Tempo trascorso: {tot_time:.3f} secondi")

In [ ]:
print(f"\nValutazione con random search iniziata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
start_time = time.time()

# Esecuzione della random search.
random_stats = random_search_cv(out_directory, Xtrain, Ytrain)
random_best_params_idx = random_stats['Mean error'].idxmin()

# Visualizzazione della miglior combinazione di iper-parametri.
print(f"\nMiglior combinazione:\n\tIndex = {random_best_params_idx},\n\tEta minus = {random_stats['Eta minus'][random_best_params_idx]:.5f},\n\tEta plus = {random_stats['Eta plus'][random_best_params_idx]:.5f},\n\tHidden layer = {random_stats['Hidden layer'][random_best_params_idx]:.0f}\n")

# Visualizzazione del tempo impiegato.
end_time = time.time()
tot_time = end_time - start_time

print(f"\nValutazione con random search completata: {datetime.now().strftime(constants.PRINT_DATE_TIME_FORMAT)}")
print(f"Tempo trascorso: {tot_time:.3f} secondi")

In [ ]:
# Recupero dei migliori iper-parametri tra grid search e random search.
# ...

# Addestramento di una nuova rete neurale su questi iper-parametri.
# ...

In [ ]:
# Verifica delle prestazioni della rete neurale sul MNIST test set
# Salvataggio delle immagini / grafici sulle predizioni.
# net.test(
#     out_directory,
#     idTest, Xtest, Ytest,
#     plot_mode=constants.PlotTestingMode.ALL
# )

# net.predict(idTest, Xtest)